In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os
import math
import cv2
import tensorflow as tf
import random

from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.utils import *
from tensorflow.keras.applications import *
from mkmodel import makemodel

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import load_model

import numpy as np
%matplotlib inline 

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import random

category = 'XR_WRIST' # VARIABLE
SIZE = 224
EPOCHS = 50
BATCH_SIZE = 128

with tf.device("/gpu:0"):

    path = os.path.join(os.getcwd(), 'MURA-v1.1')
    train_image_paths_csv = "train_image_paths.csv"

    train_images_paths = pd.read_csv(os.path.join(path,train_image_paths_csv),
                                     dtype=str,header=None)
    train_images_paths.columns = ['image_path']

    train_images_paths['label'] = train_images_paths['image_path'].map(lambda x:'positive' if 'positive' in x else 'negative')
    train_images_paths['category']  = train_images_paths['image_path'].apply(lambda x: x.split('/')[2])  
    train_images_paths['patientId']  = train_images_paths['image_path'].apply(lambda x: x.split('/')[3].replace('patient',''))


    valid_image_paths_csv = "valid_image_paths.csv"
    valid_data_paths = pd.read_csv(os.path.join(path,valid_image_paths_csv),
                                   dtype=str,header=None)
    valid_data_paths.columns = ['image_path']
    valid_data_paths['label'] = valid_data_paths['image_path'].map(lambda x:'positive' if 'positive' in x else 'negative')
    valid_data_paths['category']  = valid_data_paths['image_path'].apply(lambda x: x.split('/')[2])  
    valid_data_paths['dir'] =  valid_data_paths['image_path'].apply(lambda x: x.split('/')[1])
    valid_data_paths['patientId']  = valid_data_paths['image_path'].apply(lambda x: x.split('/')[3].replace('patient',''))

    train_mask = train_images_paths['category'] == category
    valid_mask = valid_data_paths['category']== category
    train_df_of_cat = train_images_paths
    valid_df_of_cat = valid_data_paths

    x = list()
    for idx in range(len(train_df_of_cat)):
        if train_df_of_cat["category"][idx] == category:
            x.append(train_df_of_cat.loc[idx])        
    random.shuffle(x)
    x_train = pd.DataFrame(x[:1000])

    x = list()
    for idx in range(len(valid_df_of_cat)):
        if valid_df_of_cat["category"][idx] == category:
            x.append(valid_df_of_cat.loc[idx])        
    random.shuffle(x)
    x_valid = pd.DataFrame(x[:300])

    train_df_of_cat = x_train
    valid_df_of_cat = x_valid

    train_df_of_cat = train_df_of_cat.reset_index()
    valid_df_of_cat = x_valid.reset_index()

    x = list()
    y = list()
    for idx in range(len(train_df_of_cat)):
        temp = cv2.imread(train_df_of_cat['image_path'][idx])
        resize = cv2.resize(temp, dsize=(SIZE, SIZE), interpolation=cv2.INTER_AREA)    
        x.append(resize)

    x_train = np.array(x)
    y = [0 if x=='positive' else 1 for x in train_df_of_cat['label']]
    y_train = np.array(y)

    x = list()
    y = list()
    for idx in range(len(valid_df_of_cat)):
        temp = cv2.imread(valid_df_of_cat['image_path'][idx])
        resize = cv2.resize(temp, dsize=(SIZE, SIZE), interpolation=cv2.INTER_AREA)    
        x.append(resize)    
    random.shuffle(x)
    x_test = np.array(x[:300])
    y = [0 if x=='positive' else 1 for x in valid_df_of_cat['label']]
    y_test = np.array(y[:300])



In [ ]:

from sklearn.model_selection import train_test_split

ax_train, x_tst, ay_train, y_tst = train_test_split(x_train, y_train, test_size=0.6, stratify=y_train, random_state = 1)
bx_train, cx_train, by_train, cy_train = train_test_split(x_tst, y_tst, test_size = 0.5)
#  ax_train = 0.7*0.8, x_test = 0.3*0.8
#bx_train, cx_train, by_train ,cy_train = train_test_split(ax_test, ay_test, test_size=0.5)

# print(len(ax_train))
# print(len(bx_train))
# print(len(cx_train))


x_train_passed_a = list()
x_train_passed_b = list()
x_train_passed_c = list()
input_server = np.array([])
from tensorflow.keras.optimizers import SGD

input_server_ = list()
import scipy.sparse as sp
import torch
model_base = makemodel.model_base(bx_train[0].shape)
model_base.compile(optimizer=SGD(learning_rate = 0.001),loss='binary_crossentropy', metrics=['accuracy'])

mergedmodel = Model(inputs=model_base.input, outputs=model_base.get_layer('layer').output)
# t = [0, 0, 0]
# st = time.time()
x_train_passed_a = mergedmodel.predict(ax_train)
x_train_passed_b = mergedmodel.predict(bx_train)
x_train_passed_c = mergedmodel.predict(cx_train)

x_test_passed = mergedmodel.predict(x_test)


input_server = np.concatenate((x_train_passed_b, x_train_passed_c))  #axis 1 is out of bounds for array of dimension 1
# input_server = np.concatenate((input_server,x_train_passed_b), axis=1)
input_server = np.concatenate((input_server,x_train_passed_a))

y_server = np.concatenate((by_train, cy_train))
y_server = np.concatenate((y_server, ay_train))

model_prop = makemodel.model_server(x_train_passed_b[0].shape)
model_prop.compile(optimizer=SGD(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

# hist_base = model_prop.fit(x_train_passed_c, cy_train, batch_size = BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

hist_prop = model_prop.fit(input_server, y_server, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer (Conv2D)               (None, 224, 224, 3)       84        
_________________________________________________________________
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 25089     
Total params: 20,049,557
Trainable params: 20,049,557
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_____________________________________________

In [4]:
# def load_image(img_path, target_size):
#     img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
#     img_tensor = tf.keras.preprocessing.image.img_to_array(img)

#     # 배치 사이즈 추가 + 스케일링 결과 반환
#     return img_tensor[np.newaxis] / 255     # (1, 150, 150, 3)

# def show_first_feature_map(loaded_model, img_path):
#     first_output = loaded_model.layers[1].output
#     print(first_output.shape, first_output.dtype)   # (?, 148, 148, 32) <dtype: 'float32'>

#     # 1개의 출력을 갖는 새로운 모델 생성
#     model = tf.keras.models.Model(inputs=loaded_model.input, outputs=first_output)

#     # 입력으로부터 높이와 너비를 사용해서 target_size에 해당하는 튜플 생성
#     target_size = (loaded_model.input.shape[1], loaded_model.input.shape[2])
#     img_tensor = load_image(img_path, target_size)
#     print(target_size)
#     print(loaded_model.input.shape)     # (?, 150, 150, 3)
#     print(img_tensor.shape)             # (1, 150, 150, 3)

#     first_activation = model.predict(img_tensor)

#     # 컨볼루션 레이어에서 필터 크기(3), 스트라이드(1), 패딩(valid)을 사용했기 때문에
#     # 150에서 148로 크기가 일부 줄었음을 알 수 있다. 필터 개수는 32.
#     print(first_activation.shape)
#     # print(first_activation[0, 0, 0])

#     # 19번째 활성 맵 출력. 기본 cmap은 viridis. gray는 흑백 컬러맵.
#     # [0, :, :, feature_index]
#     # 0은 첫 번째 데이터(원본 이미지)의 피처맵을 가리킨다. 사진은 1장만 사용했기 때문에 0만 가능
#     # 가운데 콜론(:)은 높이와 너비를 가리키는 차원의 모든 데이터
#     # feature_index는 보고 싶은 피처맵이 있는 채널을 가리킨다.
#     # 32개의 필터를 사용했다면 0부터 31까지의 피처맵이 존재한다.
    
#     """
#     plt.figure(figsize=(16, 16))
#     print("CHECK",first_activation.shape[-1])
#     for i in range(first_activation.shape[-1]):
#         plt.subplot(16, 16, i + 1)

#         # 눈금 제거. fignum은 같은 피켜에 연속 출력
#         plt.axis('off')
#         plt.matshow(first_activation[0, :, :, i], cmap='gray', fignum=0)
#     """
#     """
#     plt.axis('off')
#     # plt.matshow(first_activation[0, :, :, 20], cmap='gray', fignum=0)    
#     plt.matshow(first_activation[0, :, :, 20], fignum=0)    
#     plt.tight_layout()
#     plt.show()
#     """
    
#     for i in [7,18,19,27]: # 227 245   7 18 19 27
#         # plt.title(i)
#         plt.axis('off')
#         plt.matshow(first_activation[0, :, :, i], cmap='gray', fignum=0)
#         plt.tight_layout()
#         plt.show()
    
# data_dir = os.path.join(os.getcwd(), 'MURA-v1.1')
# train_dir = os.path.join(data_dir)
    
# feat_dir = os.path.join(data_dir, 'train/XR_FOREARM/patient00063/study1_positive/image1.png')
# show_first_feature_map(model_base, feat_dir)

In [ ]:
# feat_dir = os.path.join(data_dir, 'train/XR_FOREARM/patient00063/study1_positive/image1.png')
# feat_img = cv2.imread(feat_dir)
# feat_img = cv2.resize(feat_img, (280,280))
# plt.axis('off')
# plt.imshow(feat_img)
# plt.show()
# cv2.imwrite('mura_original.png', feat_img)

In [ ]:
acc_prop = hist_prop.history['val_accuracy']
loss_prop = hist_prop.history['val_loss']
df_acc = pd.DataFrame(acc_prop)
df_loss = pd.DataFrame(loss_prop)
df_acc.to_csv('mura_finger_acc_111.csv')
df_loss.to_csv('mura_finger_loss_111.csv')

In [ ]:
from sklearn.model_selection import train_test_split

ax_train, x_tst, ay_train, y_tst = train_test_split(x_train, y_train, test_size=0.3, stratify=y_train, random_state = 1)
bx_train, cx_train, by_train, cy_train = train_test_split(x_tst, y_tst, test_size = 0.3333)
#  ax_train = 0.7*0.8, x_test = 0.3*0.8
#bx_train, cx_train, by_train ,cy_train = train_test_split(ax_test, ay_test, test_size=0.5)

# print(len(ax_train))
# print(len(bx_train))
# print(len(cx_train))


x_train_passed_a = list()
x_train_passed_b = list()
x_train_passed_c = list()
input_server = np.array([])
from tensorflow.keras.optimizers import SGD

input_server_ = list()
import scipy.sparse as sp
import torch
model_base = makemodel.model_base(bx_train[0].shape)
model_base.compile(optimizer=SGD(learning_rate = 0.001),loss='binary_crossentropy', metrics=['accuracy'])

mergedmodel = Model(inputs=model_base.input, outputs=model_base.get_layer('layer').output)
# t = [0, 0, 0]
# st = time.time()
x_train_passed_a = mergedmodel.predict(ax_train)
x_train_passed_b = mergedmodel.predict(bx_train)
x_train_passed_c = mergedmodel.predict(cx_train)

x_test_passed = mergedmodel.predict(x_test)


input_server = np.concatenate((x_train_passed_b, x_train_passed_c))  #axis 1 is out of bounds for array of dimension 1
# input_server = np.concatenate((input_server,x_train_passed_b), axis=1)
input_server = np.concatenate((input_server,x_train_passed_a))

y_server = np.concatenate((by_train, cy_train))
y_server = np.concatenate((y_server, ay_train))

model_prop = makemodel.model_server(x_train_passed_b[0].shape)
model_prop.compile(optimizer=SGD(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

# hist_base = model_prop.fit(x_train_passed_c, cy_train, batch_size = BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

hist_prop = model_prop.fit(input_server, y_server, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

acc_prop = hist_prop.history['val_accuracy']
loss_prop = hist_prop.history['val_loss']
df_acc = pd.DataFrame(acc_prop)
df_loss = pd.DataFrame(loss_prop)
df_acc.to_csv('mura_finger_acc_721.csv')
df_loss.to_csv('mura_finger_loss_721.csv')


In [ ]:
from sklearn.model_selection import train_test_split

ax_train, x_tst, ay_train, y_tst = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state = 1)
bx_train, cx_train, by_train, cy_train = train_test_split(x_tst, y_tst, test_size = 0.5)
#  ax_train = 0.7*0.8, x_test = 0.3*0.8
#bx_train, cx_train, by_train ,cy_train = train_test_split(ax_test, ay_test, test_size=0.5)

# print(len(ax_train))
# print(len(bx_train))
# print(len(cx_train))


x_train_passed_a = list()
x_train_passed_b = list()
x_train_passed_c = list()
input_server = np.array([])
from tensorflow.keras.optimizers import SGD

input_server_ = list()
import scipy.sparse as sp
import torch
model_base = makemodel.model_base(bx_train[0].shape)
model_base.compile(optimizer=SGD(learning_rate = 0.001),loss='binary_crossentropy', metrics=['accuracy'])

mergedmodel = Model(inputs=model_base.input, outputs=model_base.get_layer('layer').output)
# t = [0, 0, 0]
# st = time.time()
x_train_passed_a = mergedmodel.predict(ax_train)
x_train_passed_b = mergedmodel.predict(bx_train)
x_train_passed_c = mergedmodel.predict(cx_train)

x_test_passed = mergedmodel.predict(x_test)


input_server = np.concatenate((x_train_passed_b, x_train_passed_c))  #axis 1 is out of bounds for array of dimension 1
# input_server = np.concatenate((input_server,x_train_passed_b), axis=1)
input_server = np.concatenate((input_server,x_train_passed_a))

y_server = np.concatenate((by_train, cy_train))
y_server = np.concatenate((y_server, ay_train))

model_prop = makemodel.model_server(x_train_passed_b[0].shape)
model_prop.compile(optimizer=SGD(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

# hist_base = model_prop.fit(x_train_passed_c, cy_train, batch_size = BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

hist_prop = model_prop.fit(input_server, y_server, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

acc_prop = hist_prop.history['val_accuracy']
loss_prop = hist_prop.history['val_loss']
df_acc = pd.DataFrame(acc_prop)
df_loss = pd.DataFrame(loss_prop)
df_acc.to_csv('mura_finger_acc_811.csv')
df_loss.to_csv('mura_finger_loss_811.csv')


In [ ]:
from sklearn.model_selection import train_test_split

ax_train, x_tst, ay_train, y_tst = train_test_split(x_train, y_train, test_size=0.75, stratify=y_train, random_state = 1)
bx_train, x_teest, by_train, y_teest = train_test_split(x_tst, y_tst, test_size = 0.6666)
cx_train, dx_train, cy_train, dy_train = train_test_split(x_teest, y_teest, test_size = 0.5)
#  ax_train = 0.7*0.8, x_test = 0.3*0.8


# dx_train, ex_train, dy_train ,ey_train = train_test_split(dx_train, dy_train, test_size=0.5)
# print(len(ax_train))
# print(len(bx_train))
# print(len(cx_train))


x_train_passed_a = list()
x_train_passed_b = list()
x_train_passed_c = list()
x_train_passed_d = list()
input_server = np.array([])
from tensorflow.keras.optimizers import SGD

input_server_ = list()
import scipy.sparse as sp
import torch
model_base = makemodel.model_base(bx_train[0].shape)
model_base.compile(optimizer=SGD(learning_rate = 0.001),loss='binary_crossentropy', metrics=['accuracy'])

mergedmodel = Model(inputs=model_base.input, outputs=model_base.get_layer('layer').output)
# t = [0, 0, 0]
# st = time.time()
x_train_passed_a = mergedmodel.predict(ax_train)
x_train_passed_b = mergedmodel.predict(bx_train)
x_train_passed_c = mergedmodel.predict(cx_train)
x_train_passed_d = mergedmodel.predict(dx_train)
x_test_passed = mergedmodel.predict(x_test)


input_server = np.concatenate((x_train_passed_b, x_train_passed_c))  #axis 1 is out of bounds for array of dimension 1
# input_server = np.concatenate((input_server,x_train_passed_b), axis=1)
input_server = np.concatenate((input_server,x_train_passed_a))

y_server = np.concatenate((by_train, cy_train))
y_server = np.concatenate((y_server, ay_train))

model_prop = makemodel.model_server(x_train_passed_b[0].shape)
model_prop.compile(optimizer=SGD(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

# hist_base = model_prop.fit(x_train_passed_c, cy_train, batch_size = BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

hist_prop = model_prop.fit(input_server, y_server, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

acc_prop = hist_prop.history['val_accuracy']
loss_prop = hist_prop.history['val_loss']
df_acc = pd.DataFrame(acc_prop)
df_loss = pd.DataFrame(loss_prop)
df_acc.to_csv('mura_finger_acc_1111.csv')
df_loss.to_csv('mura_finger_loss_1111.csv')


In [ ]:
from sklearn.model_selection import train_test_split

ax_train, x_tst, ay_train, y_tst = train_test_split(x_train, y_train, test_size=0.6, stratify=y_train, random_state = 1)
bx_train, x_teest, by_train, y_teest = train_test_split(x_tst, y_tst, test_size = 0.5)
cx_train, dx_train, cy_train, dy_train = train_test_split(x_teest, y_teest, test_size = 0.3333)
#  ax_train = 0.7*0.8, x_test = 0.3*0.8
#bx_train, cx_train, by_train ,cy_train = train_test_split(ax_test, ay_test, test_size=0.5)

# print(len(ax_train))
# print(len(bx_train))
# print(len(cx_train))


x_train_passed_a = list()
x_train_passed_b = list()
x_train_passed_c = list()
x_train_passed_d = list()
input_server = np.array([])
from tensorflow.keras.optimizers import SGD

input_server_ = list()
import scipy.sparse as sp
import torch
model_base = makemodel.model_base(bx_train[0].shape)
model_base.compile(optimizer=SGD(learning_rate = 0.001),loss='binary_crossentropy', metrics=['accuracy'])

mergedmodel = Model(inputs=model_base.input, outputs=model_base.get_layer('layer').output)
# t = [0, 0, 0]
# st = time.time()
x_train_passed_a = mergedmodel.predict(ax_train)
x_train_passed_b = mergedmodel.predict(bx_train)
x_train_passed_c = mergedmodel.predict(cx_train)
x_train_passed_d = mergedmodel.predict(dx_train)

x_test_passed = mergedmodel.predict(x_test)


input_server = np.concatenate((x_train_passed_b, x_train_passed_c))  #axis 1 is out of bounds for array of dimension 1
# input_server = np.concatenate((input_server,x_train_passed_b), axis=1)
input_server = np.concatenate((input_server,x_train_passed_a))

y_server = np.concatenate((by_train, cy_train))
y_server = np.concatenate((y_server, ay_train))

model_prop = makemodel.model_server(x_train_passed_b[0].shape)
model_prop.compile(optimizer=SGD(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

# hist_base = model_prop.fit(x_train_passed_c, cy_train, batch_size = BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

hist_prop = model_prop.fit(input_server, y_server, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

acc_prop = hist_prop.history['val_accuracy']
loss_prop = hist_prop.history['val_loss']
df_acc = pd.DataFrame(acc_prop)
df_loss = pd.DataFrame(loss_prop)
df_acc.to_csv('mura_finger_acc_4321.csv')
df_loss.to_csv('mura_finger_loss_4321.csv')


In [ ]:
from sklearn.model_selection import train_test_split

ax_train, x_tst, ay_train, y_tst = train_test_split(x_train, y_train, test_size=0.3, stratify=y_train, random_state = 1)
bx_train, x_teest, by_train, y_teest = train_test_split(x_tst, y_tst, test_size = 0.6666)
cx_train, dx_train, cy_train, dy_train = train_test_split(x_teest, y_teest, test_size = 0.5)
#  ax_train = 0.7*0.8, x_test = 0.3*0.8
#bx_train, cx_train, by_train ,cy_train = train_test_split(ax_test, ay_test, test_size=0.5)

# print(len(ax_train))
# print(len(bx_train))
# print(len(cx_train))


x_train_passed_a = list()
x_train_passed_b = list()
x_train_passed_c = list()
x_train_passed_d = list()
input_server = np.array([])
from tensorflow.keras.optimizers import SGD

input_server_ = list()
import scipy.sparse as sp
import torch
model_base = makemodel.model_base(bx_train[0].shape)
model_base.compile(optimizer=SGD(learning_rate = 0.001),loss='binary_crossentropy', metrics=['accuracy'])

mergedmodel = Model(inputs=model_base.input, outputs=model_base.get_layer('layer').output)
# t = [0, 0, 0]
# st = time.time()
x_train_passed_a = mergedmodel.predict(ax_train)
x_train_passed_b = mergedmodel.predict(bx_train)
x_train_passed_c = mergedmodel.predict(cx_train)
x_train_passed_d = mergedmodel.predict(dx_train)
x_test_passed = mergedmodel.predict(x_test)


input_server = np.concatenate((x_train_passed_b, x_train_passed_c))  #axis 1 is out of bounds for array of dimension 1
# input_server = np.concatenate((input_server,x_train_passed_b), axis=1)
input_server = np.concatenate((input_server,x_train_passed_a))

y_server = np.concatenate((by_train, cy_train))
y_server = np.concatenate((y_server, ay_train))

model_prop = makemodel.model_server(x_train_passed_b[0].shape)
model_prop.compile(optimizer=SGD(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

# hist_base = model_prop.fit(x_train_passed_c, cy_train, batch_size = BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

hist_prop = model_prop.fit(input_server, y_server, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

acc_prop = hist_prop.history['val_accuracy']
loss_prop = hist_prop.history['val_loss']
df_acc = pd.DataFrame(acc_prop)
df_loss = pd.DataFrame(loss_prop)
df_acc.to_csv('mura_finger_acc_7111.csv')
df_loss.to_csv('mura_finger_loss_7111.csv')


In [ ]:
from sklearn.model_selection import train_test_split

qx_train, cx_test, qy_train, cy_test = train_test_split(x_train, y_train, test_size = 0.6)
ax_train, bx_train, ay_train ,by_train = train_test_split(qx_train, qy_train, test_size=0.5)

cx_train, dx_train, cy_train ,dy_train = train_test_split(cx_test, cy_test, test_size=0.6666)

dx_train, ex_train, dy_train ,ey_train = train_test_split(dx_train, dy_train, test_size=0.5)
#  ax_train = 0.7*0.8, x_test = 0.3*0.8
#bx_train, cx_train, by_train ,cy_train = train_test_split(ax_test, ay_test, test_size=0.5)

# print(len(ax_train))
# print(len(bx_train))
# print(len(cx_train))


x_train_passed_a = list()
x_train_passed_b = list()
x_train_passed_c = list()
x_train_passed_d = list()
x_train_passed_e = list()
input_server = np.array([])
from tensorflow.keras.optimizers import SGD

input_server_ = list()
import scipy.sparse as sp
import torch
model_base = makemodel.model_base(bx_train[0].shape)
model_base.compile(optimizer=SGD(learning_rate = 0.001),loss='binary_crossentropy', metrics=['accuracy'])

mergedmodel = Model(inputs=model_base.input, outputs=model_base.get_layer('layer').output)
# t = [0, 0, 0]
# st = time.time()
x_train_passed_a = mergedmodel.predict(ax_train)
x_train_passed_b = mergedmodel.predict(bx_train)
x_train_passed_c = mergedmodel.predict(cx_train)
x_train_passed_d = mergedmodel.predict(dx_train)
x_train_passed_e = mergedmodel.predict(ex_train)

x_test_passed = mergedmodel.predict(x_test)


input_server = np.concatenate((x_train_passed_b, x_train_passed_c))  #axis 1 is out of bounds for array of dimension 1
# input_server = np.concatenate((input_server,x_train_passed_b), axis=1)
input_server = np.concatenate((input_server,x_train_passed_a))

y_server = np.concatenate((by_train, cy_train))
y_server = np.concatenate((y_server, ay_train))

model_prop = makemodel.model_server(x_train_passed_b[0].shape)
model_prop.compile(optimizer=SGD(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

# hist_base = model_prop.fit(x_train_passed_c, cy_train, batch_size = BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

hist_prop = model_prop.fit(input_server, y_server, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

acc_prop = hist_prop.history['val_accuracy']
loss_prop = hist_prop.history['val_loss']
df_acc = pd.DataFrame(acc_prop)
df_loss = pd.DataFrame(loss_prop)
df_acc.to_csv('mura_finger_acc_11111.csv')
df_loss.to_csv('mura_finger_loss_11111.csv')


In [ ]:
from sklearn.model_selection import train_test_split

qx_train, cx_test, qy_train, cy_test = train_test_split(x_train, y_train, test_size = 0.4)
ax_train, bx_train, ay_train ,by_train = train_test_split(qx_train, qy_train, test_size=0.3333)

cx_train, dx_train, cy_train ,dy_train = train_test_split(cx_test, cy_test, test_size=0.5)

dx_train, ex_train, dy_train ,ey_train = train_test_split(dx_train, dy_train, test_size=0.5)
#  ax_train = 0.7*0.8, x_test = 0.3*0.8
#bx_train, cx_train, by_train ,cy_train = train_test_split(ax_test, ay_test, test_size=0.5)

# print(len(ax_train))
# print(len(bx_train))
# print(len(cx_train))


x_train_passed_a = list()
x_train_passed_b = list()
x_train_passed_c = list()
x_train_passed_d = list()
x_train_passed_e = list()
input_server = np.array([])
from tensorflow.keras.optimizers import SGD

input_server_ = list()
import scipy.sparse as sp
import torch
model_base = makemodel.model_base(bx_train[0].shape)
model_base.compile(optimizer=SGD(learning_rate = 0.001),loss='binary_crossentropy', metrics=['accuracy'])

mergedmodel = Model(inputs=model_base.input, outputs=model_base.get_layer('layer').output)
# t = [0, 0, 0]
# st = time.time()
x_train_passed_a = mergedmodel.predict(ax_train)
x_train_passed_b = mergedmodel.predict(bx_train)
x_train_passed_c = mergedmodel.predict(cx_train)
x_train_passed_d = mergedmodel.predict(dx_train)
x_train_passed_e = mergedmodel.predict(ex_train)
x_test_passed = mergedmodel.predict(x_test)


input_server = np.concatenate((x_train_passed_b, x_train_passed_c))  #axis 1 is out of bounds for array of dimension 1
# input_server = np.concatenate((input_server,x_train_passed_b), axis=1)
input_server = np.concatenate((input_server,x_train_passed_a))

y_server = np.concatenate((by_train, cy_train))
y_server = np.concatenate((y_server, ay_train))

model_prop = makemodel.model_server(x_train_passed_b[0].shape)
model_prop.compile(optimizer=SGD(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

# hist_base = model_prop.fit(x_train_passed_c, cy_train, batch_size = BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

hist_prop = model_prop.fit(input_server, y_server, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

acc_prop = hist_prop.history['val_accuracy']
loss_prop = hist_prop.history['val_loss']
df_acc = pd.DataFrame(acc_prop)
df_loss = pd.DataFrame(loss_prop)
df_acc.to_csv('mura_finger_acc_42211.csv')
df_loss.to_csv('mura_finger_loss_42211.csv')


In [ ]:
from sklearn.model_selection import train_test_split

qx_train, cx_test, qy_train, cy_test = train_test_split(x_train, y_train, test_size = 0.3)
ax_train, bx_train, ay_train ,by_train = train_test_split(qx_train, qy_train, test_size=0.1428)

cx_train, dx_train, cy_train ,dy_train = train_test_split(cx_test, cy_test, test_size=0.6666)

dx_train, ex_train, dy_train ,ey_train = train_test_split(dx_train, dy_train, test_size=0.5)
#  ax_train = 0.7*0.8, x_test = 0.3*0.8
#bx_train, cx_train, by_train ,cy_train = train_test_split(ax_test, ay_test, test_size=0.5)

# print(len(ax_train))
# print(len(bx_train))
# print(len(cx_train))


x_train_passed_a = list()
x_train_passed_b = list()
x_train_passed_c = list()
x_train_passed_d = list()
x_train_passed_e = list()

input_server = np.array([])
from tensorflow.keras.optimizers import SGD

input_server_ = list()
import scipy.sparse as sp
import torch
model_base = makemodel.model_base(bx_train[0].shape)
model_base.compile(optimizer=SGD(learning_rate = 0.001),loss='binary_crossentropy', metrics=['accuracy'])

mergedmodel = Model(inputs=model_base.input, outputs=model_base.get_layer('layer').output)
# t = [0, 0, 0]
# st = time.time()
x_train_passed_a = mergedmodel.predict(ax_train)
x_train_passed_b = mergedmodel.predict(bx_train)
x_train_passed_c = mergedmodel.predict(cx_train)
x_train_passed_d = mergedmodel.predict(dx_train)
x_train_passed_e = mergedmodel.predict(ex_train)
x_test_passed = mergedmodel.predict(x_test)


input_server = np.concatenate((x_train_passed_b, x_train_passed_c))  #axis 1 is out of bounds for array of dimension 1
# input_server = np.concatenate((input_server,x_train_passed_b), axis=1)
input_server = np.concatenate((input_server,x_train_passed_a))

y_server = np.concatenate((by_train, cy_train))
y_server = np.concatenate((y_server, ay_train))

model_prop = makemodel.model_server(x_train_passed_b[0].shape)
model_prop.compile(optimizer=SGD(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

# hist_base = model_prop.fit(x_train_passed_c, cy_train, batch_size = BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

hist_prop = model_prop.fit(input_server, y_server, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_passed, y_test))

acc_prop = hist_prop.history['val_accuracy']
loss_prop = hist_prop.history['val_loss']
df_acc = pd.DataFrame(acc_prop)
df_loss = pd.DataFrame(loss_prop)
df_acc.to_csv('mura_finger_acc_61111.csv')
df_loss.to_csv('mura_finger_loss_61111.csv')
